In [20]:
# from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F

import pytz
# print(__name__)builtins
if __name__ == "builtins":
    spark = SparkSession \
        .builder \
        .appName("Python Demo") \
        .config("hive.metastore.client.factory.class",
                "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport() \
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    spark.conf.set("spark.sql.crossJoin.enabled", "true")
#     dd_siteid_summary_report
    dd_siteid_summary_report=spark.sql("""
        select aa.date,aa.site_id,a.register,main_page_uv,product_count_people,product_count,download_count_people,
            download_count,af_install,
            case when main_page_uv = 0 then 0 else round(product_count_people/main_page_uv,3) end as product_count_rate,
            case when main_page_uv = 0 then 0 else round(download_count_people/main_page_uv,3) end as product_download_rate,
            case when product_count_people = 0 then 0 else round(product_count/product_count_people,3) end as per_product_count,
            case when download_count_people = 0 then 0 else round(download_count/download_count_people,3) end as per_download_count
            from
            (
            select date,site_id,sum(kk) as t
            from
            (
            select date(a.create_time+ INTERVAL 7 HOUR) as date,site_id,count(a.id)as kk
            from `dumarket_s3_etl`.t_customer a 
            where a.site_id != 'STANDARD' and a.site_id != '' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR),site_id

            union

            select date(a.create_time+ INTERVAL 7 HOUR) as date,site_id,count(a.id) as kk
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !=''and a.site_id != '' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),site_id

            union

            select  date(a.create_time+ INTERVAL 7 HOUR) as date,site_id,count(a.id) as kk
            from `dumarket_s3_etl`.t_channel_install_details a 
            where a.site_id != 'STANDARD'  and a.site_id != '' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR),site_id

            )aa
            group by date,site_id
            order by date,site_id

            )aa


            left join
            (
            select date(a.create_time+ INTERVAL 7 HOUR) as date,site_id,count(a.id)as register
            from `dumarket_s3_etl`.t_customer a 
            where a.site_id != 'STANDARD' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR),site_id
            )a
            on (aa.date = a.date and aa.site_id = a.site_id)


            left join
            (
            select a.date,site_id,
            count(a.h5_id,if(a.main_page_pv>0,true ,null)) as main_page_uv,
            count(a.h5_id,if(a.detailed_page_uv>0,true ,null))  as product_count_people,sum(a.detailed_page_uv) as product_count,
            count(a.h5_id,if(a.download_button_uv>0,true ,null)) as download_count_people,sum(a.download_button_uv) as download_count
            from
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,site_id,h5_id,
            count(a.id,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),site_id,h5_id
            )a
            group by a.date,site_id
            )b
            on (aa.date = b.date and aa.site_id = b.site_id)

            left join
            (
            select  date(a.create_time+ INTERVAL 7 HOUR) as date,site_id,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a 
            where a.site_id != 'STANDARD'  and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR),site_id
            )c 
            on(aa.date = c.date and aa.site_id = c.site_id)
            order by aa.date,aa.site_id
    """)
    dd_siteid_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/dd_siteid_summary_report")
    #     dd_total_summary_report
    dd_total_summary_report=spark.sql("""
        select date,af_channel,media_source,campaign,fb_campaign_id,agency,install,register,main_page_uv,product_count_people,product_count,download_count_people,
            download_count,af_install,
            case when main_page_uv = 0 then 0 else round(product_count_people/main_page_uv,3) end as product_count_rate,
            case when main_page_uv = 0 then 0 else round(download_count_people/main_page_uv,3) end as product_download_rate,
            case when product_count_people = 0 then 0 else round(product_count/product_count_people,3) end as per_product_count,
            case when download_count_people = 0 then 0 else round(download_count/download_count_people,3) end as per_download_count
            from
            (
            select a.*,
            case when b.install is null then 0 else b.install end as install,
            case when c.register is null then 0 else c.register end as register,
            case when d.main_page_uv is null then 0 else d.main_page_uv end as main_page_uv,
            case when d.product_count_people is null then 0 else d.product_count_people end as product_count_people,
            case when d.product_count is null then 0 else d.product_count end as product_count,
            case when d.download_count_people is null then 0 else d.download_count_people end as download_count_people,
            case when d.download_count is null then 0 else d.download_count end as download_count,
            case when e.af_install is null then 0 else e.af_install end as af_install
            from 
            (
            select a.date,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,concat(date,if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency)) as tt
            from 
            (
            select '1' as t, date(date)
            from `dumarket_s3_etl`.t_date_range
            where date(date) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR) 
            )a
            left join 
            (
            select '1' as t,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,count(*) as m 
            from `dumarket_s3_etl`.t_channel_details b
            group by b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency
            )b
            on a.t = b.t
            )a
            left join 
            (
            select date(b.create_time + INTERVAL 7 HOUR) as date,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            concat(date(b.create_time + INTERVAL 7 HOUR),if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency)) as tt,
                         count(distinct a.gaid)as install
            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            where date(b.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time + INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            concat(date(b.create_time + INTERVAL 7 HOUR),if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency))
            )b
            on a.tt = b.tt
            left join 
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
                         count(distinct a.id)as register
            from `dumarket_s3_etl`.t_customer a 
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )c
            on a.tt = c.tt
            left join
            (
            select a.date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.date),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
            count(a.gaid,if(a.main_page_pv>0,true ,null)) as main_page_uv,
            count(a.gaid,if(a.detailed_page_uv>0,true ,null))  as product_count_people,sum(a.detailed_page_uv) as product_count,
            count(a.gaid,if(a.download_button_uv>0,true ,null)) as download_count_people,sum(a.download_button_uv) as download_count

            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),gaid
            )a
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            group by a.date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.date),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )d
            on a.tt = d.tt
            left join
            (select  date(a.create_time + INTERVAL 7 HOUR) as date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
                         count(a.id)as af_install
            from `dumarket_s3_etl`.t_channel_install_details a 
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )e
            on a.tt = e.tt
            )aa
            where install+register+main_page_uv+product_count_people+product_count+download_count_people+download_count+af_install>0
            order by date,af_channel,media_source,campaign,fb_campaign_id,agency
    """)
    dd_total_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/dd_total_summary_report")
     #     dd_total_summary_new_report
    dd_total_summary_new_report=spark.sql("""
       select *
            from
            (
            select date(b.create_time+ INTERVAL 7 HOUR) as date,'0' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) = date_add(b.create_time+ INTERVAL 7 HOUR ,0))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date = date_add(b.create_time+ INTERVAL 7 HOUR ,0))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date = date_add(b.create_time+ INTERVAL 7 HOUR ,0))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'1' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,2))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,2))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,2))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'2' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,3))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,3))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,3))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all



            select date(b.create_time+ INTERVAL 7 HOUR) as date,'3' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,4))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,4))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,4))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'4' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,5))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,5))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,5))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'5' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,6))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,6))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,6))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'6' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,7))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,7))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,7))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'7' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,8))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,8))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,8))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all


            select date(b.create_time+ INTERVAL 7 HOUR) as date,'15' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,16))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,16))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,16))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency


            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'30' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid and date(c.create_time+ INTERVAL 7 HOUR) < date_add(b.create_time+ INTERVAL 7 HOUR ,31))
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid and d.date < date_add(b.create_time+ INTERVAL 7 HOUR ,31))

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid and e.date < date_add(b.create_time+ INTERVAL 7 HOUR ,31))

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency

            union all

            select date(b.create_time+ INTERVAL 7 HOUR) as date,'till-now' as perfrom_day,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            count(a.gaid)as install,count(c.gaid) as register,
            count(d.gaid,if(d.main_page_pv>0,true ,null)) as main_page_uv,
            count(d.gaid,if(d.detailed_page_uv>0,true ,null))  as product_count_people,sum(d.detailed_page_uv) as product_count,
            count(d.gaid,if(d.download_button_uv>0,true ,null)) as download_count_people,sum(d.download_button_uv) as download_count,
            count(e.gaid,if(e.af_install>0,true ,null)) as af_install,
            round(count(c.gaid)/count(a.gaid),3) as regsiter_rate,

            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.detailed_page_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_count_rate,
            case when count(d.gaid,if(d.main_page_pv>0,true ,null)) = 0 then 0 else round(count(d.gaid,if(d.download_button_uv>0,true ,null))/count(d.gaid,if(d.main_page_pv>0,true ,null)),3) end as product_download_rate,
            case when count(d.gaid,if(d.detailed_page_uv>0,true ,null)) =0 then 0 else round(sum(d.detailed_page_uv) /count(d.gaid,if(d.detailed_page_uv>0,true ,null)),3) end as per_product_count,
            case when count(d.gaid,if(d.download_button_uv>0,true ,null)) = 0 then 0 else round(sum(d.download_button_uv)/count(d.gaid,if(d.download_button_uv>0,true ,null)),3) end as per_download_count


            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid     
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            left join `dumarket_s3_etl`.t_customer c on (a.gaid = c.gaid )
            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )d
            on (a.gaid = d.gaid)

            left join
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,count(a.id) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            where a.site_id = 'STANDARD' and gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid
            )e
            on (a.gaid = e.gaid)

            where date(b.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-90)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time+ INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency



            )aa
            order by date,perfrom_day,af_channel,media_source,campaign,fb_campaign_id,agency
    """)
    dd_total_summary_new_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/dd_total_summary_new_report")
     #     behavior_report
    behavior_report=spark.sql("""
       select *
            from
                 (
                  select date,'0' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,0)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                  select date,'1' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,1)
                      group by date(b.create_time)

                  )a
                  group by date

                 union all

                 select date,'2' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,2)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                  select date,'3' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,3)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all


                 select date,'4' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,4)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                  select date,'5' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                          `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,5)
                      group by date(b.create_time)

                  )a
                  group by date

                 union all

                 select date,'6' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,6)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                  select date,'7' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,7)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                 select date,'10' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,10)
                      group by date(b.create_time)

                  )a
                  group by date

                  union all

                  select date,'15' as registerday,
                         max(case when a.countName = 'user' then a.count else 0 end) as register,
                         max(case when a.countName = 'deu' then a.count else 0 end) as detailed_page_uv,
                         max(case when a.countName = 'dep' then a.count else 0 end) as detailed_page_pv,
                         max(case when a.countName = 'dou' then a.count else 0 end) as download_button_uv,
                         max(case when a.countName = 'dop' then a.count else 0 end) as download_button_pv,
                         max(case when a.countName = 'mp' then a.count else 0 end) as main_page_pv,
                         max(case when a.countName = 'mu' then a.count else 0 end) as main_page_uv,
                         max(case when a.countName = 'af' then a.count else 0 end) as af_install

                  from
                  (
                      select date(create_time) as date, 'user' as countName, count(*) as count
                      from `dumarket_s3_etl`.t_customer
                      where date(create_time) between date_add(now(), -30)  and date(now())
                      group by date(create_time)

                      union all

                      select date(b.create_time) AS date, 'af' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_channel_install_details a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dep' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'deu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_impression a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dop' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'dou' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_click a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mp' as countName, count(1) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                      union all

                      SELECT date(b.create_time) AS date, 'mu' as countName, count(distinct a.gaid) as count
                      from `dumarket_s3_etl`.t_track_detail a,
                           `dumarket_s3_etl`.t_customer b
                      where a.gaid = b.gaid
                        and a.gaid != ''
                        and date(b.create_time) between date_add(now(), -30)  and date(now())
                        and date(a.create_time) = date_add(b.create_time ,15)
                      group by date(b.create_time)

                  )a
                  group by date

                 )a
            order by date,registerday desc
    """)
    behavior_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/behavior_report")
#     operation_user_daily_report
    operation_user_daily_report=spark.sql("""
       select a.date,a.accumulation_register_acount,b.install_account,c.register_account,round(c.register_account/b.install_account,3) as register_rate,
            d.UV,d.new_UV,d.old_UV,d.bounce_rate,d.bounce_rate_of_new_visitors,e.yesterday_visited_retention_rates,
            f.detail_page_UV,f.new_detail_page_UV,f.old_detail_page_UV,f.click_product_account,f.new_click_product_account,f.old_click_product_account,
            round(f.new_click_product_account/d.new_UV,3) as new_per_product_click,round(f.old_click_product_account/d.old_UV,3) old_per_product_click
            from
            (
            select a.date,a.register+b.before_customer_count as accumulation_register_acount
            from
            (
            select b.date,sum(a.count) as register,'1' as id from (select date(create_time + INTERVAL 7 HOUR) as date,count(id)  as count from `dumarket_s3_etl`.t_customer where date(create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) group by date(create_time + INTERVAL 7 HOUR)) a,(select date(create_time + INTERVAL 7 HOUR) as date,count(id)  as count from `dumarket_s3_etl`.t_customer where date(create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) group by date(create_time + INTERVAL 7 HOUR)) b where a.date<=b.date group by b.date order by date
            )a
            left join (select count(id) as before_customer_count,'1' as id from `dumarket_s3_etl`.t_customer where date(create_time + INTERVAL 7 HOUR) < date_add(now() + INTERVAL 7 HOUR,-30))b
            on a.id = b.id
            )a
            left join
            (
            select date(create_time + INTERVAL 7 HOUR) as date,count(id)  as install_account from `dumarket_s3_etl`.t_channel_details where date(create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) group by date(create_time + INTERVAL 7 HOUR)
            )b
            on a.date = b.date
            left join
            (
            select date(create_time + INTERVAL 7 HOUR) as date,count(id) as register_account from `dumarket_s3_etl`.t_customer where date(create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) group by date(create_time + INTERVAL 7 HOUR)
            )
            c
            on a.date=c.date
            left join
            (
            select date,count(gaid) as UV,count(gaid,if(date=register_time,true,null)) as new_UV,count(gaid,if(date>register_time,true,null)) as old_UV,
            round(count(gaid,if(main_page_count=1,true,null))/count(gaid),3) as bounce_rate,
            round(count(gaid,if(main_page_count=1 and date=register_time,true,null))/count(gaid,if(date=register_time,true,null)),3) as bounce_rate_of_new_visitors
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.gaid,date(b.create_time + INTERVAL 7 HOUR) as register_time,count(a.id) main_page_count
            from `dumarket_s3_etl`.t_track_site a
            left join `dumarket_s3_etl`.t_customer b
            on a.gaid = b.gaid
            where date(a.create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(a.create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) and a.gaid !=''
            group by date(a.create_time + INTERVAL 7 HOUR),a.gaid,date(b.create_time + INTERVAL 7 HOUR)
            )main
            group by date
            )d
            on a.date=d.date
            left join
            (
            select date1 as date,round(count(gaid,if(date2!='',true,null))/count(gaid),3) as yesterday_visited_retention_rates
            from
            (
            select aa.date as date1,bb.date as date2,aa.gaid,aa.count as count1,bb.count as count2
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.gaid,count(a.id) as count
            from `dumarket_s3_etl`.t_track_site a
            where date(a.create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(a.create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) and a.gaid !=''
            group by date(a.create_time + INTERVAL 7 HOUR),a.gaid
            )aa
            left join
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.gaid,count(a.id) as count
            from `dumarket_s3_etl`.t_track_site a
            where date(a.create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-31) and date(a.create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) and a.gaid !=''
            group by date(a.create_time + INTERVAL 7 HOUR),a.gaid
            )bb
            on(aa.gaid = bb.gaid and aa.date=date_add(bb.date,1))
            )visit
            group by date1
            )e
            on a.date=e.date
            left join
            (
            select date,count(gaid) as detail_page_UV,count(gaid,if(date=register_time,true,null)) as new_detail_page_UV,count(gaid,if(date>register_time,true,null)) as old_detail_page_UV,
            sum(product_time) as click_product_account,sum(if(date=register_time,product_time,0)) as new_click_product_account,sum(if(date>register_time,product_time,0)) as old_click_product_account
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.gaid,date(b.create_time + INTERVAL 7 HOUR) as register_time,count(a.id) as click_time,count(distinct a.product_id) product_time
            from `dumarket_s3_etl`.t_track_site a
            left join `dumarket_s3_etl`.t_customer b
            on a.gaid = b.gaid
            where date(a.create_time + INTERVAL 7 HOUR) >= date_add(now() + INTERVAL 7 HOUR,-30) and date(a.create_time + INTERVAL 7 HOUR) < date(now() + INTERVAL 7 HOUR) and a.gaid !='' and type = 'IMPRESSION'
            group by date(a.create_time + INTERVAL 7 HOUR),a.gaid,date(b.create_time + INTERVAL 7 HOUR)
            )detail
            group by date
            )f
            on a.date=f.date
            order by a.date
    """)
    operation_user_daily_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/operation_user_daily_report")
    #     total_summary_report
    total_summary_report=spark.sql("""
       select date(a.create_time + INTERVAL 7 HOUR) as date,a.af_channel,a.media_source,a.campaign,a.fb_campaign_id, a.agency,
             count(distinct a.gaid)as install,count(distinct b.gaid) as register,
             count(c.gaid) as main_page_uv,sum(c.main_page_pv) as main_page_pv,
             sum(c.detailed_page_pv) as detailed_page_pv,sum(c.detailed_page_uv) as detailed_page_uv,
             sum(c.download_button_pv) as download_button_pv,sum(c.download_button_uv) as download_button_uv,sum(d.af) as af_install
            from (select * from `dumarket_s3_etl`.t_channel_details where date(create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR, -30 )  and date(now() + INTERVAL 7 HOUR) and event_name = 'install') a
            left join (select * from `dumarket_s3_etl`.t_customer where date(create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR, -30)  and date(now() + INTERVAL 7 HOUR))b on (a.gaid = b.gaid and date(a.create_time + INTERVAL 7 HOUR) = date(b.create_time + INTERVAL 7 HOUR))
            left join
            (
            select date(create_time + INTERVAL 7 HOUR) as date,gaid,count(id,if(type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(id,if(type in ('IMPRESSION'),true,null)) as detailed_page_pv,count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(id,if(type in ('CLICK_OPEN'),true,null)) as download_button_pv,count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site
            where date(create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR, -30)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),gaid
            )c on (a.gaid = c.gaid and date(a.create_time + INTERVAL 7 HOUR) = c.date)
            left join
            (select date(create_time + INTERVAL 7 HOUR) as date,gaid,count(1) as af from `dumarket_s3_etl`.t_channel_install_details where date(create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR, -30)  and date(now() + INTERVAL 7 HOUR) group by date(create_time + INTERVAL 7 HOUR),gaid )d
            on (a.gaid = d.gaid and date(a.create_time + INTERVAL 7 HOUR) = d.date)
            where a.gaid != ''
            group by date(a.create_time + INTERVAL 7 HOUR),a.af_channel,a.media_source,a.campaign,a.fb_campaign_id, a.agency
            order by date(a.create_time + INTERVAL 7 HOUR),a.af_channel,a.media_source,a.campaign,a.fb_campaign_id, a.agency
    """)
    total_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/total_summary_report")
    #     product_app_summary
    product_app_summary=spark.sql("""
      select a.date,c.media_source,a.product_id,e.name as product_name,
            count(a.gaid,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.gaid,if(a.download_button>0,true ,null)) as download_count_people,
            count(d.gaid) as af_install,
            case when count(a.gaid,if(a.detailed_page>0,true ,null)) > 0 then round(count(a.gaid,if(a.download_button>0,true ,null))/count(a.gaid,if(a.detailed_page>0,true ,null)),3) else 0 end as product_count_to_downloand_rate,
            case when count(a.gaid,if(a.download_button>0,true ,null)) > 0 then round(count(d.gaid)/count(a.gaid,if(a.download_button>0,true ,null)),3) else 0 end as dowloand_count_to_afinstall_rate

            from
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,product_id,
            count(a.id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(a.id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid,product_id
            )a
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            left join
            (
            select date(a.create_time+ INTERVAL 7 HOUR) as date,a.gaid,a.product_name,c.id,count(a.gaid) as m
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR) ,a.gaid,product_name,c.id 
            )d
            on (a.gaid = d.gaid and a.date = d.date and a.product_id = d.id)
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            where a.detailed_page>0
            group by a.date,c.media_source,a.product_id,e.name
            order by a.date,c.media_source,a.product_id
    """)
    product_app_summary.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/product_app_summary")
     #     product_channel_summary_report
    product_channel_summary_report=spark.sql("""
      select * from
            (

            select a.date,a.product_id,a.product_name,'site_id' as type,a.site_id as channel,
            case when b.product_count_people is null then 0 else b.product_count_people end as product_count_people,
            case when b.download_count_people is null then 0 else b.download_count_people end as download_count_people,
            case when c.af_install is null then 0 else c.af_install end as af_install, 
            case when (b.product_count_people is null or b.download_count_people is null) then 0 when (b.product_count_people = 0 or b.download_count_people = 0)  then 0 else round(b.download_count_people/b.product_count_people,3) end as product_count_to_downloand_rate,
            case when (b.download_count_people is null or c.af_install is null) then 0 when (b.download_count_people = 0 or c.af_install = 0)  then 0 else round(c.af_install/b.download_count_people,3) end as dowloand_count_to_afinstall_rate

            from
            (
            select date,site_id,product_id,product_name,sum(kk) as t
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(a.id) as kk
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 

            union 

            select a.date,site_id,product_id,e.name as product_name,count(detailed_page) as kk
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )f
            group by date,site_id,product_id,product_name
            )a


            left join
            (
            select a.date,site_id,product_id,e.name as product_name,
            count(a.h5_id,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.h5_id,if(a.download_button>0,true ,null)) as download_count_people
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )b
            on (a.date=b.date and a.site_id= b.site_id and a.product_id = b.product_id and a.product_name = b.product_name)

            left join
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(distinct a.gaid) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 
            )c
            on (a.date=c.date and a.site_id= c.site_id and a.product_id = c.product_id and a.product_name = c.product_name)

            where a.t > 0

            union all

            select a.date,a.product_id,e.name as product_name,'media_source' as type,c.media_source as channel,
            count(a.gaid,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.gaid,if(a.download_button>0,true ,null)) as download_count_people,
            count(d.gaid) as af_install,
            case when count(a.gaid,if(a.detailed_page>0,true ,null)) > 0 then round(count(a.gaid,if(a.download_button>0,true ,null))/count(a.gaid,if(a.detailed_page>0,true ,null)),3) else 0 end as product_count_to_downloand_rate,
            case when count(a.gaid,if(a.download_button>0,true ,null)) > 0 then round(count(d.gaid)/count(a.gaid,if(a.download_button>0,true ,null)),3) else 0 end as dowloand_count_to_afinstall_rate

            from
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,product_id,
            count(a.id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(a.id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid,product_id
            )a
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            left join
            (
            select date(a.create_time+ INTERVAL 7 HOUR) as date,a.gaid,a.product_name,c.id,count(a.gaid) as m
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR) ,a.gaid,product_name,c.id 
            )d
            on (a.gaid = d.gaid and a.date = d.date and a.product_id = d.id)
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            where a.detailed_page>0
            group by a.date,c.media_source,a.product_id,e.name

            )a
            order by date,product_id,type,channel
    """)
    product_channel_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/product_channel_summary_report")
     #     product_siteid_summary_report
    product_siteid_summary_report=spark.sql("""
      select a.date,a.site_id,a.product_id,a.product_name,
            case when b.product_count_people is null then 0 else b.product_count_people end as product_count_people,
            case when b.download_count_people is null then 0 else b.download_count_people end as download_count_people,
            case when c.af_install is null then 0 else c.af_install end as af_install, 
            case when (b.product_count_people is null or b.download_count_people is null) then 0 when (b.product_count_people = 0 or b.download_count_people = 0)  then 0 else round(b.download_count_people/b.product_count_people,3) end as product_count_to_downloand_rate,
            case when (b.download_count_people is null or c.af_install is null) then 0 when (b.download_count_people = 0 or c.af_install = 0)  then 0 else round(c.af_install/b.download_count_people,3) end as dowloand_count_to_afinstall_rate

            from
            (
            select date,site_id,product_id,product_name,sum(kk) as t
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(a.id) as kk
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 

            union 

            select a.date,site_id,product_id,e.name as product_name,count(detailed_page) as kk
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )f
            group by date,site_id,product_id,product_name
            )a


            left join
            (
            select a.date,site_id,product_id,e.name as product_name,
            count(a.h5_id,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.h5_id,if(a.download_button>0,true ,null)) as download_count_people
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )b
            on (a.date=b.date and a.site_id= b.site_id and a.product_id = b.product_id and a.product_name = b.product_name)

            left join
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(distinct a.gaid) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 
            )c
            on (a.date=c.date and a.site_id= c.site_id and a.product_id = c.product_id and a.product_name = c.product_name)

            where a.t > 0
            order by a.date,a.site_id,a.product_id,a.product_name
    """)
    product_siteid_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/product_siteid_summary_report")
    #     product_type_summary_report
    product_type_summary_report=spark.sql("""
      select date,product_id,product_name,type,sum(product_count_people) as product_count_people,sum(download_count_people) as download_count_people,sum(af_install) as af_install,
            case when (sum(product_count_people) >  0 and sum(download_count_people) > 0 ) then  round(sum(download_count_people)/sum(product_count_people),3) else 0 end as product_count_to_downloand_rate,
            case when  (sum(download_count_people)> 0 and sum(af_install)> 0) then  round(sum(af_install)/sum(download_count_people),3) else 0 end as dowloand_count_to_afinstall_rate


            from
            (

            select a.date,a.product_id,a.product_name,'site_id' as type,a.site_id as channel,
            case when b.product_count_people is null then 0 else b.product_count_people end as product_count_people,
            case when b.download_count_people is null then 0 else b.download_count_people end as download_count_people,
            case when c.af_install is null then 0 else c.af_install end as af_install

            from
            (
            select date,site_id,product_id,product_name,sum(kk) as t
            from
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(a.id) as kk
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 

            union 

            select a.date,site_id,product_id,e.name as product_name,count(detailed_page) as kk
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )f
            group by date,site_id,product_id,product_name
            )a


            left join
            (
            select a.date,site_id,product_id,e.name as product_name,
            count(a.h5_id,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.h5_id,if(a.download_button>0,true ,null)) as download_count_people
            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,site_id,h5_id,product_id,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id != 'STANDARD' and a.h5_id !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),site_id,h5_id,product_id
            )a
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            group by a.date,site_id,product_id,name
            )b
            on (a.date=b.date and a.site_id= b.site_id and a.product_id = b.product_id and a.product_name = b.product_name)

            left join
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,a.site_id,c.id as product_id,a.product_name,count(distinct a.gaid) as af_install
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id != 'STANDARD' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now() + INTERVAL 7 HOUR,-60)  and date(now() + INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),a.site_id,c.id,product_name 
            )c
            on (a.date=c.date and a.site_id= c.site_id and a.product_id = c.product_id and a.product_name = c.product_name)

            where a.t > 0

            union all

            select a.date,a.product_id,e.name as product_name,'media_source' as type,c.media_source as channel,
            count(a.gaid,if(a.detailed_page>0,true ,null))  as product_count_people,
            count(a.gaid,if(a.download_button>0,true ,null)) as download_count_people,
            count(d.gaid) as af_install

            from
            (
            select date(create_time+ INTERVAL 7 HOUR) as date,gaid,product_id,
            count(a.id,if(type in ('IMPRESSION'),true,null)) as  detailed_page,
            count(a.id,if(type in ('CLICK_OPEN'),true,null)) as  download_button
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time+ INTERVAL 7 HOUR),gaid,product_id
            )a
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            left join
            (
            select date(a.create_time+ INTERVAL 7 HOUR) as date,a.gaid,a.product_name,c.id,count(a.gaid) as m
            from `dumarket_s3_etl`.t_channel_install_details a
            left join `dumarket_s3_etl`.t_loan_product c
            on a.product_name = c.name
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time+ INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time+ INTERVAL 7 HOUR) ,a.gaid,product_name,c.id 
            )d
            on (a.gaid = d.gaid and a.date = d.date and a.product_id = d.id)
            left join `dumarket_s3_etl`.t_loan_product e
            on a.product_id = e.id
            where a.detailed_page>0
            group by a.date,c.media_source,a.product_id,e.name

            )a
            group by date,product_id,product_name,type
            order by date,product_id,type
    """)
    product_type_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/product_type_summary_report")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
dd_total_summary_report=spark.sql("""
        select date,af_channel,media_source,campaign,fb_campaign_id,agency,install,register,main_page_uv,product_count_people,product_count,download_count_people,
            download_count,af_install,
            case when main_page_uv = 0 then 0 else round(product_count_people/main_page_uv,3) end as product_count_rate,
            case when main_page_uv = 0 then 0 else round(download_count_people/main_page_uv,3) end as product_download_rate,
            case when product_count_people = 0 then 0 else round(product_count/product_count_people,3) end as per_product_count,
            case when download_count_people = 0 then 0 else round(download_count/download_count_people,3) end as per_download_count
            from
            (
            select a.*,
            case when b.install is null then 0 else b.install end as install,
            case when c.register is null then 0 else c.register end as register,
            case when d.main_page_uv is null then 0 else d.main_page_uv end as main_page_uv,
            case when d.product_count_people is null then 0 else d.product_count_people end as product_count_people,
            case when d.product_count is null then 0 else d.product_count end as product_count,
            case when d.download_count_people is null then 0 else d.download_count_people end as download_count_people,
            case when d.download_count is null then 0 else d.download_count end as download_count,
            case when e.af_install is null then 0 else e.af_install end as af_install
            from 
            (
            select a.date,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,concat(date,if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency)) as tt
            from 
            (
            select '1' as t, date(date)
            from `dumarket_s3_etl`.t_date_range
            where date(date) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR) 
            )a
            left join 
            (
            select '1' as t,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,count(*) as m 
            from `dumarket_s3_etl`.t_channel_details b
            group by b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency
            )b
            on a.t = b.t
            )a
            left join 
            (
            select date(b.create_time + INTERVAL 7 HOUR) as date,b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            concat(date(b.create_time + INTERVAL 7 HOUR),if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency)) as tt,
                         count(distinct a.gaid)as install
            from 
            (
            select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details where gaid is not null group by gaid
            )a
            left join `dumarket_s3_etl`.t_channel_details b on a.id = b.id
            where date(b.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(b.create_time + INTERVAL 7 HOUR),b.af_channel,b.media_source,b.campaign,b.fb_campaign_id, b.agency,
            concat(date(b.create_time + INTERVAL 7 HOUR),if(b.af_channel is null,1,b.af_channel),if(b.media_source is null ,2,b.media_source),if(b.campaign is null,3,b.campaign),if(b.fb_campaign_id is null ,4,b.fb_campaign_id),if(b.agency is null,5,b.agency))
            )b
            on a.tt = b.tt
            left join 
            (
            select date(a.create_time + INTERVAL 7 HOUR) as date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
                         count(distinct a.id)as register
            from `dumarket_s3_etl`.t_customer a 
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )c
            on a.tt = c.tt
            left join
            (
            select a.date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.date),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
            count(a.gaid,if(a.main_page_pv>0,true ,null)) as main_page_uv,
            count(a.gaid,if(a.detailed_page_uv>0,true ,null))  as product_count_people,sum(a.detailed_page_uv) as product_count,
            count(a.gaid,if(a.download_button_uv>0,true ,null)) as download_count_people,sum(a.download_button_uv) as download_count

            from
            (
            select date(create_time + INTERVAL 7 HOUR) as date,gaid,
            count(a.gaid,if(a.type in ('MAIN_PAGE_DEFAULT','MAIN_PAGE_AMOUNT_DESC','MAIN_PAGE_RATE_ASC'),true,null)) as main_page_pv,
            count(distinct product_id,if(type in ('IMPRESSION'),true,null)) as  detailed_page_uv,
            count(distinct product_id,if(type in ('CLICK_OPEN'),true,null)) as  download_button_uv
            from `dumarket_s3_etl`.t_track_site a 
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(create_time + INTERVAL 7 HOUR),gaid
            )a
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            group by a.date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.date),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )d
            on a.tt = d.tt
            left join
            (select  date(a.create_time + INTERVAL 7 HOUR) as date,c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency)) as tt,
                         count(a.id)as af_install
            from `dumarket_s3_etl`.t_channel_install_details a 
            left join 
            (select gaid,min(id) as id from `dumarket_s3_etl`.t_channel_details group by gaid )b
            on a.gaid = b.gaid
            left join `dumarket_s3_etl`.t_channel_details c
            on b.id = c.id
            where a.site_id = 'STANDARD' and a.gaid !='' and date(a.create_time + INTERVAL 7 HOUR) between date_add(now()+ INTERVAL 7 HOUR,-60)  and date(now()+ INTERVAL 7 HOUR)
            group by date(a.create_time + INTERVAL 7 HOUR),c.af_channel,c.media_source,c.campaign,c.fb_campaign_id, c.agency,
            concat(date(a.create_time + INTERVAL 7 HOUR),if(c.af_channel is null,1,c.af_channel),if(c.media_source is null ,2,c.media_source),if(c.campaign is null,3,c.campaign),if(c.fb_campaign_id is null ,4,c.fb_campaign_id),if(c.agency is null,5,c.agency))
            )e
            on a.tt = e.tt
            )aa
            where install+register+main_page_uv+product_count_people+product_count+download_count_people+download_count+af_install>0
            order by date,af_channel,media_source,campaign,fb_campaign_id,agency
    """)
dd_total_summary_report.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/stream/dumarket_etl/dd_total_summary_report")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'Detected implicit cartesian product for LEFT OUTER join between logical plans\nProject [cast(date#41 as date) AS date#225]\n+- Filter (isnotnull(date#41) && ((cast(date#41 as date) >= 18421) && (cast(date#41 as date) <= 18481)))\n   +- Relation[id#40L,date#41,year#42,month#43,day#44,date_str#45] orc\nand\nAggregate [af_channel#48, media_source#49, campaign#55, fb_campaign_id#57, agency#56], [af_channel#48, media_source#49, campaign#55, fb_campaign_id#57, agency#56]\n+- Project [af_channel#48, media_source#49, campaign#55, agency#56, fb_campaign_id#57]\n   +- Relation[id#46L,channel_key#47,af_channel#48,media_source#49,event_name#50,body#51,ip_address#52,create_time#53,fb_campaign_name#54,campaign#55,agency#56,fb_campaign_id#57,gaid#58] orc\nJoin condition is missing or trivial.\nEither: use the CROSS JOIN syntax to allow cartesian products between these\nrelations, or: enable implicit cartesian products by setting the configuration\nvariable spark.sql.crossJoin.enabled=true;'
Tracebac

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+---------+-----+
|date|countName|count|
+----+---------+-----+
+----+---------+-----+